In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import chisquare, iqr
from scipy import signal

# Get amplifier data from text file
def getAmpData(fileDir, fileName):
    freqs = []
    amps = []
    with open(fileDir + fileName, 'r') as f:
        for line in f:
            holder = line.split()
            freqs.append(float(holder[0])/1000.)
            amps.append(float(holder[1]))
    return np.asarray(freqs), np.asarray(amps)

# Define a Gaussian function
def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

# Label setup for plots
label_font = {'fontname':'Arial', 'size':'22', 'color':'black', 'weight':'normal',
          'verticalalignment':'bottom'} 
title_font = {'fontname':'sans-serif', 'size':'16', 'color':'black', 'weight':'bold'}
legend_font = {'family':'sans-serif', 'size':'20', 'style':'normal'} 
tickSize = 16
labelpadVal = 30


This Jupyter notebook will load in and analyze the data taken by the Spectran V6 spectrum analyzer. By this time, you should have successfully taken about 1 day of data that has been saved in a .CSV file. You will now analyze these data to look candidate dark photon signals and set a limit on the dark photon coupling strength, $\varepsilon$
over the frequency span you took data. We start by first loading this data into a dataframe.

In [2]:
#Set the directory and file name where your data are stored. DataDir is the relative
#path. If you put the data in the same directory as this notebook leave it as "./"
dataDir = './Data/'
fileName = 'groupA_4p1GHzCF_122MHzSpan_quarterSpan_6_23_23_1240to1400.csv'
ampInfoDir = './'
# Load the data in the .CSV into a tabular data structure called a pandas dataframe. This will allow 
# you to easily index individual acquisitions and add them together. The first index gives the
# frequency. The second index is a time index. All values are given in 'dBm'. This can be converted to
# Watts via P_dBm = 10log10(P_W/1 mW)

dataDF = pd.read_csv(dataDir + fileName)

#print some shape info
print(f'{dataDF.shape[0]} time steps and {dataDF.shape[1]} frequency bins')

#print(f'{round(dataDF['time'][dataDF.shape[0]-1] - dataDF['time'][0], 2)} seconds')
#Display the first few rows and columns
dataDF.head()


90 time steps and 6805 frequency bins


,time,4087242505.034009,4087246255.032529,4087250005.031050,4087253755.029570,4087257505.028090,4087261255.026610,4087265005.025131,4087268755.023651,4087272505.022171,...,4112719994.980788,4112723744.979308,4112727494.977829,4112731244.976349,4112734994.974869,4112738744.973390,4112742494.971910,4112746244.970430,4112749994.968950,4112753744.967471
0,0.653227,-91.35,-91.33,-91.32,-91.27,-91.33,-91.31,-91.27,-91.29,-91.32,...,-91.71,-91.65,-91.63,-91.68,-91.66,-91.64,-91.75,-91.69,-91.69,-91.69
1,53.986319,-91.30,-91.35,-91.31,-91.29,-91.37,-91.33,-91.39,-91.37,-91.30,...,-91.72,-91.75,-91.73,-91.67,-91.74,-91.71,-91.66,-91.67,-91.67,-91.70
2,107.319674,-91.30,-91.27,-91.32,-91.34,-91.26,-91.30,-91.33,-91.32,-91.31,...,-91.63,-91.64,-91.73,-91.76,-91.67,-91.73,-91.69,-91.69,-91.62,-91.68
3,160.653006,-91.29,-91.33,-91.34,-91.27,-91.33,-91.29,-91.34,-91.27,-91.29,...,-91.66,-91.68,-91.72,-91.65,-91.71,-91.75,-91.73,-91.68,-91.68,-91.75
4,213.986334,-91.29,-91.31,-91.34,-91.28,-91.30,-91.31,-91.34,-91.32,-91.32,...,-91.73,-91.70,-91.63,-91.63,-91.69,-91.65,-91.69,-91.68,-91.66,-91.63


In [ ]:
%matplotlib widget

# Averaging all the collected data together and store as numpy array 
amps    = np.asarray([10**(np.asarray(dataDF[x])/10.).mean() for x in dataDF.columns[1:]])
ampsdBm = 10*np.log10(amps)

# Extract bin frequencies of the data collected and store and numpy array
dataFreqs = np.asarray([float(x) for x in dataDF.columns[1:]])

#Shhhh...
amps[int(len(amps)/4.12345)] *= 1.0048
print(dataFreqs[int(len(amps)/4.12345)])

#Plot

plt.plot(dataFreqs/10**9, 10*np.log10(amps))
plt.xlabel(r'Frequency (GHz)', labelpad = labelpadVal, **label_font)
plt.ylabel('Averaged Power (dBm)', **label_font)
plt.xticks(fontsize = tickSize)
plt.yticks(fontsize = tickSize)
plt.show()
plt.tight_layout()


Question: What is this spectrum showing us?

Question: How much variation (in %) do we see between the maximum 
          and minimum values?
          
Question: How would we go about determining if there is a dark photon signal
          on top of this baseline?

Question: In the empty cell below, replot this data in terms of Watts instead of dBm.

In [ ]:
plt.close('all')
###############Your code here###############
plt.plot(dataFreqs/10**9, ampsdBm, label = 'Power (dBm)')





############################################


plt.xlabel('Frequency (GHz)', labelpad = labelpadVal, **label_font)
plt.ylabel('Averaged Power (Watts)', **label_font)
plt.xticks(fontsize = tickSize)
plt.yticks(fontsize = tickSize)
plt.legend()
plt.show()
plt.tight_layout()

In [ ]:
plt.close('all')
dataLin = (amps/1000.)
plt.plot(dataFreqs/10**9, dataLin)
plt.xlabel('Frequency (GHz)', labelpad = labelpadVal, **label_font)
plt.ylabel('Power (W)', **label_font)
plt.xticks(fontsize = tickSize)
plt.yticks(fontsize = tickSize)
plt.title('Output-Referred Power vs Frequency', y = 1.08, **title_font)
plt.show()
plt.tight_layout()

This is the measured power at the Aaronia post amplification. It is also known as the **output-referred power**.
However, we would like to know the power measured by the antenna. This is known
as the **input-referred power**. The input-referred power is a combination of two parts:
The power from the antenna and the noise power due to the amplifiers. Therefore, to
turn the output-referred power to an input-referred power the following steps must be
performed:
1) Divide by the gain of the amplification chain. We have measured this for your and provide it in the 'ZX60Gain_Measured_BothAmps_5-30-22.npy' file.

2) Subtract the noise power of the amplifier. This is normally specified as a noise temperature $T$, given by $P = k_{B} \times T \times RBW$ where $P$ is the noise power, $k_{B}$ is Boltzmann's
constant, and $RBW$ is the resolution bandwidth. Note $k_{B} \sim \frac{[Energy]}{[Temperture]}$, so $k_{B} \times T$ has units of wattsper Hertz. This is known as $power spectral density$ or **PSD**. Note that noise temperture data were taken from the manufacturer data sheet and provided in 'Minicircuits_ZX60-63GLN_NoiseFigure.txt'

In [ ]:
plt.close('all')
gainFile = 'ZX60Gain_Measured_BothAmps_5-30-22.npy'
nfFile = 'Minicircuits_ZX60-63GLN_NoiseFigure.txt'

# Load in gain data
measGains = np.load(ampInfoDir + gainFile)
measGainFreqs = np.linspace(3.8, 4.2, 10000)
totalFitVals = 50

# Gain / noise temperature was taken over a larger frequency span than we care about. This narrows the displayed
# data to a region we care about.
startIndex = np.argmin(np.abs(measGainFreqs-dataFreqs[0]/10**9))
endIndex = np.argmin(np.abs(measGainFreqs-dataFreqs[-1]/10**9))
endIndex = endIndex + (totalFitVals - (endIndex - startIndex)%totalFitVals)
startIndex = startIndex - totalFitVals

# Smooth out the gain data. Compensates for some noise in the measurment set up. 
gainFitAmps = np.asarray([np.median(x) for x in measGains[startIndex:endIndex].reshape((-1, totalFitVals))])
gainFitFreqs = np.asarray([np.median(x) for x in measGainFreqs[startIndex:endIndex].reshape((-1, totalFitVals))])

# Get noise figure data and convert to a noise temperature
nfFreqs, nfVals = getAmpData(ampInfoDir, nfFile)
nfVals = (10**(nfVals/10.)-1)*300.
nfStartIndex = np.argmin(np.abs(nfFreqs-dataFreqs[0]/10**9))
nfEndIndex = np.argmin(np.abs(nfFreqs-dataFreqs[-1]/10**9))

# Plot gain data
plt.plot(gainFitFreqs, gainFitAmps, linewidth = 2, label = 'Fit')
plt.xlim([dataFreqs[0]/10**9, dataFreqs[-1]/10**9])
plt.xlabel('Frequency (GHz)', labelpad = labelpadVal, **label_font)
plt.ylabel('Gain (dB)', **label_font)
#plt.legend(prop = legend_font)
plt.show()
plt.tight_layout()

# Plot noise temperature data
plt.figure()
plt.plot(nfFreqs[nfStartIndex-5:nfEndIndex+5], nfVals[nfStartIndex-5:nfEndIndex+5])
plt.xlabel('Frequency (GHz)', labelpad = labelpadVal, **label_font)
plt.ylabel('Noise Temperature (K)', **label_font)
plt.show()
plt.tight_layout()


The above cell is taking in the gain and noise temperature files and plotting them.
Notice that the gain curve looks very bumpy, but the variation is only 0.4 dB. This is a
byproduct of the acquisition / fitting method.

Question: What is the noise power spectral density associated with a 75K noise temperature?

Question: If the resolution (RBW) is 3.75 kHz, what is the noise power we would measure due to the 75K amplifier? What if the resolution was 1 Hz? Do we measure the same noise power? Is this ok?

In [ ]:
plt.close('all')
# Linearly interpolate gain values for all frequency bins in the acquired data
gainInterp = 10**(np.interp(dataFreqs, gainFitFreqs, gainFitAmps)/10.)

# Linearly interpolate noise temperature values for all frequency bins in the acquired data
# and conver to noise power
nfInterp = np.interp(dataFreqs, nfFreqs, nfVals)*(1.381*10**-23)*3750

# Subtract off the noise power and divide by the interpolated gain
dataCorrected = dataLin / gainInterp - nfInterp

# Plot power spectral density vs. frequency
interestingIndex = int(len(amps)/4.12345)


plt.plot((dataFreqs/10**9), (10*np.log10(dataCorrected*1000/3750.)))
#plt.plot([(dataFreqs/10**9)[interestingIndex]], [((10*np.log10(dataCorrected*1000/3750.)))[interestingIndex]+ 0.05], 'ro', markersize = 10)

plt.xlabel('Frequency (GHz)', labelpad = labelpadVal, **label_font)
plt.ylabel(r'PSD (dBm/Hz)', **label_font)
plt.xticks(fontsize = tickSize)
plt.yticks(fontsize = tickSize)
plt.show()
plt.tight_layout()

The above shows the input referred PSD with the input noise temperature
of the amplifier subtracted out.

Question: What temperature does the median of the data represent (just guesstimate the median)?

Question: Does this make sense?

Question: What should the PSD be? Hint; Johnson-Nyquist noise

In [ ]:
plt.close('all')
# Sampling rate such that the total amount of data is 1s
fs = len(dataFreqs)

# Cutoff in number of bins
fc = 30

# Normalize the frequency in term of Nyquist
fcNorm = 2./(fc)

# Create a 6th-order Butterworth filter - returns numerator (b) and denominator (a) polynomials of the IIR filter
b, a = signal.butter(6, fcNorm, 'highpass', analog = False)

# Compute the frequency response of a digital filter.
# w: The frequencies at which h was computed, in the same units as fs.
# h: The frequency response, as complex numbers.
w, h = signal.freqz(b, a, worN=16384)
binNumberAxis = np.linspace(6000, 2, len(w))

# Plot the frequency response on a semilog plot
plt.semilogx(1 / (w[1:] / (2*np.pi)), 20 * np.log10(abs(h[1:])), label = 'High-Pass')
#plt.gca().invert_xaxis()
plt.margins(0, 0.1)
plt.xlim([2000, 1])
plt.gca().set_xticks([10**3, 10**2, 10**1, 10**0])
plt.grid(which='both', axis='both')
plt.axvline(fc, color='green') # cutoff frequency
plt.xlabel('Number of Bins', labelpad = labelpadVal, **label_font)
plt.ylabel('Amplitude (dB)', **label_font)
plt.xticks(fontsize = tickSize)
plt.yticks(fontsize = tickSize)
plt.show()
plt.tight_layout()

We know that dark photon signals are very narrow (only a bin wide). We can use this fact to filter the power spectrum. Assume a sampling rate of one bin per unit time. Then the narrowest signals that can be resolved are two bins wide. To minimize the chance that we attenuate signals of interest we set the corner frequency much more than a single bin. 


Question: Why is this filter called a highpass filter?

Question: Why is this filter called a passive filter?

Question: How much does a  signal get attenuated by?

In [ ]:
plt.close('all')
# Apply the Butterworth filter to the acquired data that has been corrected for the 
# noise power of the amplifiers and the gain of the amplifiers
filteredData = signal.filtfilt(b, a, dataCorrected[:])
plt.plot((dataFreqs/10**9)[:], (filteredData*10**21))
plt.xlabel('Frequency (GHz)', labelpad = labelpadVal, **label_font)
plt.ylabel(r'Filtered Power ($\times 10^{-21}$W)', **label_font)
plt.xticks(fontsize = tickSize)
plt.yticks(fontsize = tickSize)
plt.savefig('FinalFrame_SmallSignal_Filtered.png', dpi = 100, bbox_inches = 'tight')
plt.show()
plt.tight_layout()

Notice that applying the filter removes the baseline shaping of the unfiltered input referred power spectrum. However, it has also removed the DC offset. Thus, there are negative values even though the units are
still Watts. This is okay though because sensitivity to dark photons is related to the standard deviation of
this distribution not the mean. 

Question: Try setting the cutoff frequency on the filter to 10 bins. Then try setting it to 1,000. What happens? Why?

Question: Why is there an error when you set the corner frequency to 1 bin?

Set the corner frequency to 100 bins and continue with the rest of the notebook.

To see that this filtering has dramatically improved the simplicity in making significance claims, we histogram it, fit a Gaussian to the resulting distribution. By the Central Limit theorem, the data are (approximately) normally distributed. 

In [ ]:
plt.close('all')
# Scale the filtered data to more sensible unit (zW)
scaledFilteredData = filteredData*10**21


# Apply the Freedman-Diaconis rule to determine bin widths / number of bins
binWidth = 2 * iqr(scaledFilteredData) / (len(scaledFilteredData)**(1./3.))
numBins = int((max(scaledFilteredData) - min(scaledFilteredData))/binWidth)
binSpace = np.linspace(min(scaledFilteredData), max(scaledFilteredData), numBins)

# Histogram the scaled and filtered power spectrum
n, bins, patches = plt.hist(scaledFilteredData, bins = binSpace, density = False)
plt.xlabel(r'Filtered Power ($\times 10^{-21}$W)', labelpad = labelpadVal, **label_font)
plt.ylabel('Counts', **label_font)

# Fit a Gaussian to the histogramm-ed data 
popt,pcov = curve_fit(gaus,(bins[1:] + bins[:-1])/2., n, p0=[390.0,0,np.std(scaledFilteredData)])

#fitAmps = gaus((bins[1:] + bins[:-1])/2., *popt)
#fitAmps = fitAmps*np.sum(n)/np.sum(fitAmps)


# Display the fitted Gaussian
fitPows = np.linspace(min(scaledFilteredData), max(scaledFilteredData), num = 200)
fitPlots = gaus(fitPows, *popt)
fitPlotsPlus = fitPlots + np.sqrt(fitPlots)
fitPlotsMinus = fitPlots - np.sqrt(fitPlots)
plt.semilogy(fitPows, fitPlots, 'r-', linewidth = 1)
plt.fill_between(fitPows, fitPlotsPlus, fitPlotsMinus, color = 'r', alpha = .7, zorder = 1000, label = r'Fit')
#plt.plot(fitPows, fitPlotsPlus, 'r-', linewidth = 3)
#plt.plot(fitPows, fitPlotsMinus, 'r-', linewidth = 3)
#plt.gca().set_yscale('lin')
plt.legend(prop = legend_font)
plt.ylim([0.9, 450])
plt.xticks(fontsize = tickSize)
plt.yticks(fontsize = tickSize)
plt.show()
plt.tight_layout()

Question: How do we set a limit on $\varepsilon$?

We can determine a limit by setting a confidence interval. Namely, assuming that $n$ samples are drawn
from a normal distribution with the mean and standard deviation of the fitted curve, we set 
a confidence limit $C_{L}$ such that there is only a chance $p$ of seeing a variation of more
than $C_{L}$. $C_{L}$ can be defined in terms of the number of standard deviations, $\sigma$,
of the fitted Gaussian. Commonly, $p$ is set at 95%. For $n = 6804$ (the number of samples in
the spectrum), this limit is set at $4.33\sigma$. Note, we expect a dark photon to appear as a 
positive-going excursion on the baseline so do not consider negative-going signals as candidates.

With this in mind, we can now search through the data to see if there are any signals that are 4.3$\sigma$
or greater. If there are, we have found dark photon candidates! If there are not (or we can exclude candidates), we can set a 95% confidence limit over the span of the acquired data.

In [ ]:
stdVal = popt[2]
totalCandidates = 0

# Scan through data frequencies and scaled filtered data for candidates
for counter, val in enumerate(zip(dataFreqs/10**9, scaledFilteredData)):
    if val[1] > stdVal*4.33:
        totalCandidates += 1
        print(str(round(val[1]/stdVal, 3)) + u'\u03C3 CANDIDATE FOUND AT ' + str(round(val[0], 3)) + ' GHZ')
        print('THIS IS BIN NUMBER ' + str(counter))

if(totalCandidates):
    print(str(totalCandidates) + ' CANDIDATES FOUND')
else:
    print('NO CANDIDATES FOUND')
stdVal *= 10**-21

Up to now, we have been looking at power. The question is, how do we convert from power to
a dark photon coupling constant, $\varepsilon$? We know a couple of things:

1) We can relate the measured power, $P_{m}$ to an RMS voltage. Using $V = IR$ and $P=IV$ we find
$$
V_{RMS} = \sqrt{\frac{{P_{m}}}{{\lvert Z_{in}} \rvert}}
$$
where $Z_{in}$ is the input impedance of the spectrum analyzer and is approximately 50$\Omega$.


2) The measured electric field, $E_{m}$ is related to the coupling constant $\varepsilon$ via 

$$
\begin{align}
E_{m} = 3,300\,V/m \cdot \varepsilon .
\end{align}
$$




This assumes that the local dark matter energy density is $0.3\,GeV/cm^{3}$ (from astronomical observations).

However, we have measured a voltage, and $\varepsilon$ is related to the measured electrc field. How do we convert the two?

An antenna produces a voltage at its port depending on the incident field. The relationship between incident field and port voltage is called the antenna factor or $AF$ and has units of $m^{-1}$. It is defined as

$$
\begin{align} 
V = \frac{E}{AF}.
\end{align}
$$

The antenna factor is approximately given for some frequency, $\nu$, as

$$
\begin{align}
AF = \frac{9.73 \nu}{c}
\end{align}
$$

where $c$ is the speed of light. At $4\,\text{GHz}$, the $AF$ is approximately 130. We now have a relation between the measured power and the $AF$ given by

Equating V in the two equations above we find 

$$
\begin{align}
E = \sqrt{P \, \lvert Z_{in} \rvert} \cdot{AF}
\end{align}
$$

Using $E_{m} = 3,300\,V/m \cdot \varepsilon $

$$
\begin{align}
\varepsilon = \frac{\sqrt{P \, \lvert Z_{in}\rvert}}{3,300 V/m}  \cdot{AF}
\end{align}
$$

or,
$$
\begin{align}
\varepsilon = {0.28\times W^{-1/2}} \times \sqrt{P_{m}} = {0.28 W^{-1/2}} \times \sqrt{4.33\sigma_{P}}
\end{align}
$$



In [ ]:
AF = 130 #1/m
Z = 50 #ohms
E = 3300 #V/m

prefactor = np.sqrt(Z)/E * AF
print(round(prefactor,3), '1/sqrt(watts)')


In [ ]:
epsVal = 0.2785*np.sqrt(4.3*stdVal)
print("{:.3e}".format(epsVal))

In [ ]:
%run ./DPHandbookCode.ipynb

# The following code is taken from arXiv:2105.04565v3

fig,ax = FigSetup()

# DPDM
DarkMatter(ax)

# Axion haloscopes
Haloscopes(ax)

# # LSW/Helioscopes
LSW(ax)
CAST(ax)
SHIPS(ax)

# Tests of coulomb law
Coulomb(ax)

# # Reactor neutrinos
TEXONO(ax)

# # Geomagnetic field
SuperMAG(ax)

# # DPDM searches
Xenon(ax)
DAMIC(ax)
SENSEI(ax)
SuperCDMS(ax)
FUNK(ax)
LAMPOST(ax)
Tokyo(ax)
SHUKET(ax)
DarkEfield(ax)
WISPDMX(ax)
SQuAD(ax)
DMPathfinder(ax)

# # Astrophysical boundse
StellarBounds(ax)
COBEFIRAS(ax)
Jupiter(ax)
Earth(ax)
Crab(ax)
IGM(ax)
LeoT(ax)
GasClouds(ax)
NeutronStarCooling(ax)

# BHSR
plt.fill_between([6.5e-15,2.9e-11],[1e-18,1e-18],y2=1,color='gray',edgecolor='none',zorder=-100,alpha=0.25)
plt.gcf().text(0.304,0.176,r'Black hole',fontsize=23,ha='center',rotation=0,color='k',alpha=0.8)
plt.gcf().text(0.304,0.145,r'superradiance',fontsize=23,ha='center',rotation=0,color='k',alpha=0.8)


# Final label
plt.arrow(0.435, 0.375, 0, -0.055, transform=fig.transFigure,figure=fig,
  length_includes_head=True,lw=2.5,
  head_width=0.012, head_length=0.028, overhang=0.13,
  edgecolor='k',facecolor='w',clip_on=False,zorder=-1)

plt.text(4e-9,0.8e-14,r'Dark',fontsize=27,ha='center')
plt.text(4e-9,0.15e-14,r'photon',fontsize=27,ha='center')
plt.text(4e-9,0.02e-14,r'DM',fontsize=27,ha='center')


massConversion = 6.626 * 10**-34 / (1.609 * 10**-19)
massVal = np.median(dataFreqs) * massConversion

plt.plot([massVal], [epsVal], 'ko', markersize = 15)
plt.text(2*massVal, epsVal, 'HEPCAT',fontsize=16, weight = 'bold', ha='left')
plt.savefig('HEPCAT_Limits.pdf')
plt.show()